In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#our data, 4 documents

docs=["NLP consists of computational tools of textual analysis",
      "Pizza is a savory dish of Italian origin and the ingredients usually include cheese.",
      "leafy branches are swaying in the wind and leaves are gently falling in the river.",
      "The main task of machine learning is using computational methods to do prediction."
     ]

titles=["NLP","Pizza","tree","ML"]

In [ ]:
import sklearn.feature_extraction 


## Finding similarities

Which of the last three dosuments is more similar to the first doument?

In [ ]:
#CountVectorizer is a tool that count words inside each string.
#At this point we are ignoring the order of words,
#and the relationship between words.

#bag of words
bower=sklearn.feature_extraction.text.CountVectorizer()



In [ ]:
docterm=bower.fit_transform(docs) #you can also do it in two steps


In [ ]:
bower. #explore this object

In [ ]:
#dictionary of all the words in the corpus (aggregation of all documents)
vocab=bower.vocabulary_
print(len(vocab))
vocab

In [ ]:
bower.get_feature_names() 
#without building vocabulary

In [ ]:
docterm


docterm is a sparse matrix:

- majority of the elements in the matrix are zero.

- only non-zero elements are stored to help with efficiency. 

In [ ]:
print(docterm)
#rows => documents, columns=> words/terms/tokens

In [ ]:
print(docterm.toarray())   #non-sparse format

In [ ]:
docterm.toarray().shape

In [ ]:
#Build a dataframe based on our de-sparsed matrix

doctermDF = pd.DataFrame(docterm.toarray(),
                         columns=bower.get_feature_names(),
                         index=titles )
doctermDF

In [ ]:
#trasnpose the matrix (swap rows with columns)
doctermDF = pd.DataFrame(docterm.toarray().T,
                         columns= titles,
                         index=bower.get_feature_names())
doctermDF



- Words (rows) with more than one non-zero element are present in multiple documents and can be used to measure similraity.

- Word compisition of each document is available in each column.




Notice that we do not see any single-letter words in our dataframe. All the single character tokens are ignored by the default tokenizer. 

If you want single character tokens to be in the vocabulary, then you have to use a costume tokenizer

In [ ]:
sklearn.feature_extraction.text.CountVectorizer?

In [ ]:
#get the single-letter words...

bower=sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=lambda txt: txt.split())
docterm=bower.fit_transform(docs)


In [ ]:
doctermDF = pd.DataFrame(docterm.toarray().T,
                         columns= titles,
                        index=bower.get_feature_names())


doctermDF

Notice the loss of the syntactic structure when we only use the frequency of words in the document

In [ ]:

bower=sklearn.feature_extraction.text.CountVectorizer(tokenizer=lambda txt: txt.split())
docterm=bower.fit_transform(docs)


doctermDF = pd.DataFrame(docterm.toarray(),
                         columns=bower.get_feature_names(),
                         index=titles)
doctermDF

In [ ]:
from sklearn.preprocessing import normalize

df=pd.DataFrame(normalize(doctermDF),
                columns=bower.get_feature_names(),
                index=titles)
df

In [ ]:
(0.277350*0.277350)*13

In [ ]:
#let's make the first document our reference
ref=df.iloc[0]   #iloc gvies numerical location of a dataframe
cosines=df.dot(ref)
cosines.nlargest()   #sorted version

- Why do you think pizza document is somewhat similar to NLP document?

- Now, we want to remove stop words, such as "a", "the", "of", etc. and repeat our analysis.

- In some applications stop words usage might be insightful.

In [ ]:

bower=sklearn.feature_extraction.text.CountVectorizer(stop_words="english")
docterm=bower.fit_transform(docs)


doctermDF = pd.DataFrame(docterm.toarray(),columns=bower.get_feature_names(),index=titles)
doctermDF

Which words are not in the dataframe anymore?

In [ ]:
df=pd.DataFrame(normalize(doctermDF),
                columns=bower.get_feature_names(),
                index=titles)
ref=df.iloc[0]
similarities=df.dot(ref)
similarities.nlargest()

- Now we add the word "document" to all documents. So we expect to see more similarities based on word count. But, are the documents really more similar?


In [ ]:
docs=["first document: NLP consists of computational tools of textual analysis",
      "second document: Pizza is a savory dish of Italian origin and the ingredients usually include cheese.",
      "third document: leafy branches are swaying in the wind and leaves are gently falling in the river.",
      "fourth document: The main task of machine learning is using computational methods to do prediction."
     ]
 
titles=["NLP","Pizza","tree","ML"]


bower=sklearn.feature_extraction.text.CountVectorizer(
    stop_words="english")
docterm=bower.fit_transform(docs)

doctermDF = pd.DataFrame(docterm.toarray().T,columns= titles,
                        index=bower.get_feature_names())

doctermDF



In [ ]:

bower=sklearn.feature_extraction.text.CountVectorizer(stop_words="english")
docterm=bower.fit_transform(docs)


doctermDF = pd.DataFrame(docterm.toarray(),columns=bower.get_feature_names(),index=titles)
df=pd.DataFrame(normalize(doctermDF),columns=bower.get_feature_names(),index=titles)
ref=df.iloc[0]
similarities=df.dot(ref)
similarities.nlargest()

- How to give higher weight to "surprising" words?

- How to penalize the words that appear across all documents?

In [ ]:
# we use TfidfVectorizer instead of CountVectorizer
bower=sklearn.feature_extraction.text.TfidfVectorizer(
    stop_words="english")
docterm=bower.fit_transform(docs)


doctermDF = pd.DataFrame(docterm.toarray().T,
                         columns= titles,
                        index=bower.get_feature_names())

doctermDF

In [ ]:

doctermDF = pd.DataFrame(docterm.toarray(),
                         columns=bower.get_feature_names(),
                         index=titles)
df=pd.DataFrame(normalize(doctermDF),
                columns=bower.get_feature_names(),
                index=titles)
ref=df.iloc[0]
similarities=df.dot(ref)
similarities.nlargest()

# Text Preprocessing 

# NLTK

- A widely used NLP platform used in Python. 

- It's usually used for generic purposes.

In [ ]:
import nltk 

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

In [ ]:
#sample text, sourced from the wikipedia of poetry
text="""Poetry is a form of literature that 
          uses aesthetic and often rhythmic qualities of language—such as phonaesthetics,
          sound symbolism, and metre—to evoke meanings in addition to, 
          or in place of, the prosaic ostensible meaning.
          Poetry has a long history – dating back to prehistoric times with hunting poetry
          in Africa, and to panegyric and elegiac court poetry of the empires of the Nile,
          Niger, and Volta River valleys.
          """

- This text is reasonably clean, compared to user-generated text. There is no misspellings, informal spellings, etc. 

- The "-...-" phrase is challenging, and might be confused with a word that has a hyphen.

In [ ]:
#separating the tokens

nltk_tokenized = nltk.word_tokenize(text)
print(nltk_tokenized)

### Stemming

- Reducing the words to their basic forms, by trimming the ends.

- for example: teaching and teaches will be reduced to teach.

In [ ]:
#porter stemmer
porter = nltk.PorterStemmer()
porter_results = [porter.stem(t) for t in nltk_tokenized]
print(porter_results)

In [ ]:
porter.stem("teaches") ,porter.stem("teaching") ,porter.stem("teacher") 

In [ ]:
#snowball stemmer

snowball = nltk.SnowballStemmer("english")
snowball_results = [snowball.stem(t) for t in nltk_tokenized]
print(snowball_results)

### Lemmatization

- Reducing the words, to their linguistic cores.

- For example, qualities to quality.

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
lemmatizer_results= [lemmatizer.lemmatize(t) for t in nltk_tokenized]
print(lemmatizer_results)

In [ ]:
lemmatizer.lemmatize("studies"), snowball.stem("studies")

### tokenizing based on the sentences

In [ ]:
nltk_tokenized_sents = nltk.sent_tokenize(text)
print(nltk_tokenized_sents)

In [ ]:
#excluding stopwords
from nltk.corpus import stopwords
print(sorted(stopwords.words('english')))


- check the stopwords in the package you use. 

- Based on applications you might want to remove some words from the stopwords.

In [ ]:
text_noSW = [w for w in nltk_tokenized if w not in stopwords.words('english')]
print(text_noSW)

### part of speech tagging

In [ ]:

sentence_pos = nltk.pos_tag(nltk_tokenized)
print(sentence_pos)

# Spacy

- More user friendly and more off-the-shelf style


In [ ]:
import sys
!{sys.executable} -m pip install spacy

In [ ]:
!{sys.executable} -m spacy download en

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")  #English core

In [ ]:
spacy_doc= nlp(text)

In [ ]:
spacy_doc.   #explore this object

In [ ]:
print(    [w.text for w in spacy_doc]     )  #tokenized version

- Spacy could detect the hyphen phrase.

In [ ]:
print(    [w.lemma_ for w in spacy_doc]     )

Let's compare lemmatization in Spacy and NLTK.


Notice "use".

In [ ]:
print(lemmatizer_results)

### Entity Recongnition

- Automatically detecting locations, organization, etc. 

- The corpus used to feed and train the package algorithms (training body), affect the results of the algorithms.

- sometimes, we have the option to choose the training body.

In [ ]:
spacy_doc.ents

In [ ]:
ents=spacy_doc.ents
for x in ents:
    print(x.text + ": " + x.label_)

In [ ]:
sents=spacy_doc.sents
for item in sents:
    print("sentence:", item)

In [ ]:
sentence2="UMD is a public research university in College Park, Maryland."

In [ ]:
spacy_doc= nlp(sentence2)
spacy_doc.ents

In [ ]:
ents=spacy_doc.ents
for x in ents:
    print(x.text + ": " + x.label_)
    
#GPE: geo-political entity

# textblob



In [ ]:
import sys
!{sys.executable} -m pip install textblob



In [ ]:
import textblob

In [ ]:
tb_doc=textblob.TextBlob(text)

In [ ]:
print(tb_doc.words)  #couldn't specify the phrase inside hyphens.

In [ ]:
print([w.lemmatize() for w in tb_doc.words])  #notice "us"

In [ ]:
tb_doc.sentences  

The task of detecting sentences in our text was not challenging for any of the packages. For example, adding abbreviations that have dots make the sentence detection harder.

### ngram

- Higher order of organizations of words

- How to capture 'College Park" together and different from 'College' and 'Park'?

- We can add ngrams to our analysis.

In [ ]:
tb_doc.ngrams(n=2)

### noun phrases

- they can be 2grams or have more words.

- The presence and absence of phrases in different documents can help us to find similarities between them.

In [ ]:
print(tb_doc.noun_phrases)

#notice 'uses aesthetic'.

In [ ]:
tb_doc.np_counts

#### part of speech

In [ ]:
print(tb_doc.pos_tags)

In [ ]:
print(sentence_pos)  #NLTK

### Sentiment Analysis

- What sentiment the text has and what is the intensity.

- One step beyond counting the words towards the meaning and the emotional content of the text

- The simplest sentiment analysis returns the valence of the text, and assigns a number to the text. 

- Textblob return two numbers.

- Polarity can be  postitve or negative.

- Subjectivity is between zero and one.

In [ ]:
tb_doc.sentiment

In [ ]:
tb_doc.sentiment_assessments

In [ ]:
tb_doc.tokens

In [ ]:
x=textblob.TextBlob("great")
x.sentiment

In [ ]:
x=textblob.TextBlob("good")
x.sentiment

In [ ]:
x=textblob.TextBlob("not great")
x.sentiment

In [ ]:
x=textblob.TextBlob("good thing")
x.sentiment  #adding thing doesn't change polarity and subjectivity

In [ ]:
x=textblob.TextBlob("not a good thing")
x.sentiment  #a does not affect anything. #not multiplies the polarity by -0.5

In [ ]:
x=textblob.TextBlob("very")
x.sentiment  #on average very is used more in positiove sense.

In [ ]:
x=textblob.TextBlob("very good")
x.sentiment #very multiplies the subjectivity by 1.3

In [ ]:
x=textblob.TextBlob("very great")
x.sentiment

In [ ]:
x="I believe that this is a terrible sentence and a horrible example. "
tb=textblob.TextBlob(x)
tb.sentiment

In [ ]:
tb.sentiment_assessments

In [ ]:
x="I don't believe that this is a terrible sentence and certanily don't think it's a horrible example. "
tb=textblob.TextBlob(x)
tb.sentiment  #textblob fails to capture the negation

In [ ]:
x="I believe that this is not a terrible sentence and I think it's not a horrible example. "
tb=textblob.TextBlob(x)
tb.sentiment  #now, textblob captures the negation

In [ ]:
x="I believe that this is not a bad sentence and I think it's not a horrible example. "
tb=textblob.TextBlob(x)
tb.sentiment #replacing terrible with bad decreases subjectivity

In [ ]:
x="I know that this is not a bad sentence and I think it's not a horrible example. "
tb=textblob.TextBlob(x)
tb.sentiment 

In [ ]:
x="I think this is an average sentence and might work as an example. "
tb=textblob.TextBlob(x)
tb.sentiment

In [ ]:
tb.sentiment_assessments

In [ ]:
x="I think this is an above-average sentence and might work as an example. "
tb=textblob.TextBlob(x)
tb.sentiment

How the sentiment anlysis is implemented?

- Usually a large corpus is used to train the supervised machine learning problem.

- The labels are usually are given manually (binary or score). 


### NRC-emotion

- going beyond sentiment and detect emotions.

- Free and public lexicon: 
https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm
    
    

In [ ]:
import sys
!{sys.executable} -m pip install NRCLex



In [ ]:
from nrclex import NRCLex

In [ ]:
text="I like that beautiful tree"
text_object = NRCLex(text)

In [ ]:
text_object.affect_dict

In [ ]:
text="""I don't believe that this is a terrible sentence 
    and certanily don't think it's a horrible example. """
text_object = NRCLex(text)
text_object.affect_dict